In [1]:
from itertools import product
from operator import itemgetter

from PIL import Image
import random
import numpy

import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision.transforms import ToTensor

numpy.set_printoptions(threshold=numpy.nan)

In [2]:
## step 1: create test and train sets

In [3]:
def create_image(x, y):
    im = Image.new(mode='L', size=(128, 128), color=0)
    pixels = im.load()
    pixels[x, y] = 256
    return im

In [4]:
data = [(x, y) for x,y in product(range(128), range(128))]

In [5]:
random.shuffle(data)

In [6]:
train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]

In [7]:
for x,y in train_data:
    im = create_image(x, y)
    im.save("./train_set/{}_{}.bmp".format(x, y))

for x,y in test_data:
    im = create_image(x, y)
    im.save("./test_set/{}_{}.bmp".format(x, y))

In [8]:
## step 2: train the network to find radius of circle [only use train data in this stage]

In [9]:
def rotated_img(image, center):
    images = numpy.array([numpy.array(image.rotate(d, center=center)) for d in range(360)])
    arr = numpy.array(numpy.sum(images, axis=(0)), dtype=numpy.uint8)
    out = Image.fromarray(arr, mode='L')
    return out

In [10]:
class Circles(Dataset):
    def __init__(self, train_data):
        self.train_xy = train_data
        self.to_tensor = ToTensor()
    def __len__(self):
        return len(self.train_xy)
    def __getitem__(self, i):
        x, y = self.train_xy[i]
        im = Image.open("./train_set/{}_{}.bmp".format(x, y))
        #center = (random.randint(0, 127), random.randint(0, 127))
        center = random.sample([(0, 0), (128, 0), (0, 128), (128, 128)], 1)
        rotated_image = rotated_img(im, center[0])
        radius = ((x - center[0][0])**2 + (y - center[0][1])**2)**0.5
        return {
            "image": self.to_tensor(rotated_image),
            "radius": torch.LongTensor([int(radius / 185.0 * 100.0)])
        }

In [11]:
dataset = Circles(train_data)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 10, 5)
        self.conv3 = nn.Conv2d(10, 10, 3)
        self.fc1 = nn.Linear(10 * 13 * 13, 250)
        self.fc2 = nn.Linear(250, 200)
        self.fc3 = nn.Linear(200, 100)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 10 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [13]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# net.to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [15]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs
        inputs = data["image"]
        radius = data["radius"].view(data["radius"].numel())
        #inputs, radius = inputs.to(device), radius.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, radius)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            print(radius[0], torch.max(outputs, 1)[1][0])

print('Finished Training')

[1,   100] loss: 4.603
tensor(80) tensor(52)
[1,   200] loss: 4.599
tensor(70) tensor(52)
[1,   300] loss: 4.591
tensor(17) tensor(59)
[1,   400] loss: 4.584
tensor(20) tensor(45)
[1,   500] loss: 4.579
tensor(66) tensor(45)
[1,   600] loss: 4.572
tensor(50) tensor(52)
[1,   700] loss: 4.562
tensor(25) tensor(62)
[1,   800] loss: 4.548
tensor(38) tensor(62)
[2,   100] loss: 4.497
tensor(55) tensor(64)
[2,   200] loss: 4.411
tensor(46) tensor(61)
[2,   300] loss: 4.396
tensor(65) tensor(63)
[2,   400] loss: 4.381
tensor(66) tensor(68)
[2,   500] loss: 4.368
tensor(57) tensor(67)
[2,   600] loss: 4.346
tensor(76) tensor(67)
[2,   700] loss: 4.370
tensor(56) tensor(60)
[2,   800] loss: 4.361
tensor(62) tensor(67)
[3,   100] loss: 4.290
tensor(72) tensor(68)
[3,   200] loss: 4.197
tensor(31) tensor(68)
[3,   300] loss: 3.817
tensor(60) tensor(58)
[3,   400] loss: 3.062
tensor(68) tensor(65)
[3,   500] loss: 2.526
tensor(52) tensor(51)
[3,   600] loss: 2.068
tensor(63) tensor(64)
[3,   700]

In [16]:
GUESSES = [(x, y) for x, y in product(range(128), range(128))]
GUESS_R = [
    (
        ((x - 0) ** 2 + (y - 0) ** 2) ** 0.5,
        ((x - 128) ** 2 + (y - 0) ** 2) ** 0.5,
        ((x - 0) ** 2 + (y - 128) ** 2) ** 0.5
    )
    for x, y in GUESSES   
]

def get_r(im, net, center):
    to_tensor = ToTensor()
    rotated_image = to_tensor(rotated_img(im, center))
    out = net(rotated_image.unsqueeze(0))
    r = torch.max(out, 1)[1][0]
    return r.item() * 1.0 / 100.0 *  185.0

def get_x_y(im, net):
    r1 = get_r(im, net, (0, 0))
    r2 = get_r(im, net, (128, 0))
    r3 = get_r(im, net, (0, 128))
    guess_dist = [
        (r1 - r1_guess) ** 2 + (r2 - r2_guess) ** 2 + (r3 - r3_guess) ** 2
        for r1_guess, r2_guess, r3_guess in GUESS_R
    ]
    idx = min(enumerate(guess_dist), key=itemgetter(1))[0]
    return GUESSES[idx]


In [17]:
import glob

for fname in glob.glob("./test_set/*.bmp"):
    im = Image.open(fname)
    x,y = get_x_y(im, net)
    real_x, real_y = fname.split("/")[-1].split(".")[0].split("_")
    print("REAL = {}, {} .... GUESSED = {}, {}".format(real_x, real_y, x, y))


REAL = 48, 97 .... GUESSED = 48, 96
REAL = 100, 126 .... GUESSED = 2, 23
REAL = 25, 6 .... GUESSED = 25, 6
REAL = 97, 42 .... GUESSED = 97, 42
REAL = 110, 96 .... GUESSED = 110, 96
REAL = 43, 0 .... GUESSED = 43, 0
REAL = 37, 126 .... GUESSED = 38, 125
REAL = 33, 98 .... GUESSED = 32, 96
REAL = 57, 6 .... GUESSED = 57, 5
REAL = 60, 46 .... GUESSED = 59, 46
REAL = 119, 74 .... GUESSED = 118, 73
REAL = 33, 16 .... GUESSED = 34, 16
REAL = 27, 4 .... GUESSED = 27, 5
REAL = 115, 110 .... GUESSED = 18, 13
REAL = 48, 105 .... GUESSED = 48, 104
REAL = 42, 111 .... GUESSED = 42, 110
REAL = 83, 91 .... GUESSED = 83, 90
REAL = 106, 79 .... GUESSED = 105, 78
REAL = 50, 55 .... GUESSED = 50, 55
REAL = 15, 61 .... GUESSED = 16, 62
REAL = 101, 113 .... GUESSED = 100, 111
REAL = 106, 97 .... GUESSED = 106, 97
REAL = 14, 72 .... GUESSED = 13, 71
REAL = 109, 109 .... GUESSED = 109, 109
REAL = 41, 109 .... GUESSED = 40, 108
REAL = 18, 44 .... GUESSED = 19, 44
REAL = 72, 48 .... GUESSED = 71, 46
REAL = 11

REAL = 52, 5 .... GUESSED = 53, 6
REAL = 70, 60 .... GUESSED = 69, 59
REAL = 97, 65 .... GUESSED = 96, 65
REAL = 90, 107 .... GUESSED = 88, 106
REAL = 36, 67 .... GUESSED = 36, 67
REAL = 6, 115 .... GUESSED = 9, 114
REAL = 23, 126 .... GUESSED = 75, 72
REAL = 15, 66 .... GUESSED = 16, 66
REAL = 98, 81 .... GUESSED = 97, 80
REAL = 118, 107 .... GUESSED = 21, 9
REAL = 12, 123 .... GUESSED = 16, 121
REAL = 3, 86 .... GUESSED = 5, 86
REAL = 39, 125 .... GUESSED = 38, 124
REAL = 113, 44 .... GUESSED = 112, 43
REAL = 88, 122 .... GUESSED = 87, 121
REAL = 118, 23 .... GUESSED = 118, 23
REAL = 115, 81 .... GUESSED = 115, 81
REAL = 36, 10 .... GUESSED = 37, 11
REAL = 27, 26 .... GUESSED = 28, 27
REAL = 40, 23 .... GUESSED = 41, 24
REAL = 80, 42 .... GUESSED = 79, 41
REAL = 0, 28 .... GUESSED = 2, 29
REAL = 41, 62 .... GUESSED = 42, 62
REAL = 105, 29 .... GUESSED = 103, 28
REAL = 74, 26 .... GUESSED = 74, 27
REAL = 118, 15 .... GUESSED = 71, 72
REAL = 85, 72 .... GUESSED = 85, 71
REAL = 17, 63 .

REAL = 122, 74 .... GUESSED = 121, 73
REAL = 24, 61 .... GUESSED = 25, 61
REAL = 86, 51 .... GUESSED = 86, 51
REAL = 65, 71 .... GUESSED = 63, 70
REAL = 47, 105 .... GUESSED = 47, 104
REAL = 100, 76 .... GUESSED = 98, 75
REAL = 117, 31 .... GUESSED = 116, 32
REAL = 95, 35 .... GUESSED = 94, 33
REAL = 127, 120 .... GUESSED = 120, 115
REAL = 65, 6 .... GUESSED = 65, 7
REAL = 3, 83 .... GUESSED = 5, 83
REAL = 66, 114 .... GUESSED = 65, 114
REAL = 83, 119 .... GUESSED = 83, 118
REAL = 91, 83 .... GUESSED = 90, 82
REAL = 54, 57 .... GUESSED = 54, 57
REAL = 53, 15 .... GUESSED = 53, 15
REAL = 114, 104 .... GUESSED = 114, 104
REAL = 17, 92 .... GUESSED = 17, 92
REAL = 88, 23 .... GUESSED = 87, 21
REAL = 8, 107 .... GUESSED = 6, 106
REAL = 41, 115 .... GUESSED = 40, 113
REAL = 36, 111 .... GUESSED = 36, 110
REAL = 76, 54 .... GUESSED = 74, 52
REAL = 42, 92 .... GUESSED = 41, 91
REAL = 38, 89 .... GUESSED = 38, 88
REAL = 108, 90 .... GUESSED = 106, 88
REAL = 77, 94 .... GUESSED = 76, 94
REAL = 

REAL = 11, 123 .... GUESSED = 125, 125
REAL = 37, 78 .... GUESSED = 37, 77
REAL = 88, 65 .... GUESSED = 88, 64
REAL = 106, 32 .... GUESSED = 106, 31
REAL = 39, 61 .... GUESSED = 39, 62
REAL = 81, 11 .... GUESSED = 82, 13
REAL = 119, 49 .... GUESSED = 118, 47
REAL = 113, 62 .... GUESSED = 112, 61
REAL = 124, 102 .... GUESSED = 124, 102
REAL = 107, 108 .... GUESSED = 106, 108
REAL = 47, 39 .... GUESSED = 47, 39
REAL = 67, 49 .... GUESSED = 68, 49
REAL = 76, 81 .... GUESSED = 76, 81
REAL = 107, 112 .... GUESSED = 106, 111
REAL = 103, 97 .... GUESSED = 102, 96
REAL = 73, 115 .... GUESSED = 73, 113
REAL = 10, 45 .... GUESSED = 12, 45
REAL = 37, 8 .... GUESSED = 38, 9
REAL = 96, 60 .... GUESSED = 95, 59
REAL = 43, 71 .... GUESSED = 44, 71
REAL = 114, 67 .... GUESSED = 113, 67
REAL = 107, 78 .... GUESSED = 106, 77
REAL = 23, 92 .... GUESSED = 23, 92
REAL = 5, 82 .... GUESSED = 7, 83
REAL = 112, 99 .... GUESSED = 111, 98
REAL = 95, 72 .... GUESSED = 94, 71
REAL = 34, 122 .... GUESSED = 34, 121

REAL = 46, 109 .... GUESSED = 44, 107
REAL = 0, 101 .... GUESSED = 69, 64
REAL = 84, 93 .... GUESSED = 83, 92
REAL = 79, 76 .... GUESSED = 79, 75
REAL = 45, 75 .... GUESSED = 45, 75
REAL = 43, 53 .... GUESSED = 43, 54
REAL = 36, 37 .... GUESSED = 37, 37
REAL = 121, 56 .... GUESSED = 121, 55
REAL = 50, 14 .... GUESSED = 51, 14
REAL = 52, 8 .... GUESSED = 53, 9
REAL = 34, 6 .... GUESSED = 34, 8
REAL = 121, 59 .... GUESSED = 120, 58
REAL = 113, 115 .... GUESSED = 12, 15
REAL = 17, 14 .... GUESSED = 113, 112
REAL = 48, 24 .... GUESSED = 47, 24
REAL = 20, 124 .... GUESSED = 73, 73
REAL = 84, 117 .... GUESSED = 84, 116
REAL = 82, 110 .... GUESSED = 82, 109
REAL = 51, 109 .... GUESSED = 51, 109
REAL = 5, 14 .... GUESSED = 6, 14
REAL = 119, 100 .... GUESSED = 118, 99
REAL = 28, 20 .... GUESSED = 29, 20
REAL = 125, 69 .... GUESSED = 123, 68
REAL = 81, 86 .... GUESSED = 81, 86
REAL = 39, 43 .... GUESSED = 39, 43
REAL = 85, 15 .... GUESSED = 85, 15
REAL = 65, 7 .... GUESSED = 65, 7
REAL = 86, 16 

...

REAL = 92, 127 .... GUESSED = 91, 127
REAL = 45, 126 .... GUESSED = 45, 125
REAL = 93, 30 .... GUESSED = 92, 28
REAL = 106, 68 .... GUESSED = 104, 67
REAL = 11, 18 .... GUESSED = 110, 116
REAL = 78, 104 .... GUESSED = 78, 103
REAL = 53, 112 .... GUESSED = 52, 111
REAL = 57, 113 .... GUESSED = 57, 112
REAL = 49, 61 .... GUESSED = 49, 61
REAL = 122, 72 .... GUESSED = 122, 71
REAL = 37, 41 .... GUESSED = 37, 40
REAL = 0, 79 .... GUESSED = 3, 79
REAL = 91, 68 .... GUESSED = 90, 67
REAL = 107, 40 .... GUESSED = 105, 39
REAL = 89, 89 .... GUESSED = 88, 88
REAL = 126, 95 .... GUESSED = 125, 95
REAL = 15, 84 .... GUESSED = 15, 84
REAL = 69, 14 .... GUESSED = 70, 14
REAL = 112, 30 .... GUESSED = 111, 29
REAL = 26, 98 .... GUESSED = 25, 97
REAL = 83, 78 .... GUESSED = 82, 77
REAL = 70, 14 .... GUESSED = 70, 14
REAL = 20, 46 .... GUESSED = 20, 45
REAL = 90, 115 .... GUESSED = 90, 115
REAL = 33, 70 .... GUESSED = 33, 70
REAL = 27, 14 .... GUESSED = 27, 13
REAL = 18, 88 .... GUESSED = 17, 88
REAL =

REAL = 93, 12 .... GUESSED = 92, 12
REAL = 56, 78 .... GUESSED = 55, 77
REAL = 78, 87 .... GUESSED = 77, 87
REAL = 51, 53 .... GUESSED = 51, 52
REAL = 68, 68 .... GUESSED = 67, 67
REAL = 37, 59 .... GUESSED = 37, 58
REAL = 2, 78 .... GUESSED = 4, 78
REAL = 76, 45 .... GUESSED = 75, 45
REAL = 56, 3 .... GUESSED = 56, 3
REAL = 81, 33 .... GUESSED = 80, 33
REAL = 58, 80 .... GUESSED = 57, 79
REAL = 55, 21 .... GUESSED = 56, 21
REAL = 20, 7 .... GUESSED = 120, 107
REAL = 16, 99 .... GUESSED = 17, 99
REAL = 17, 94 .... GUESSED = 17, 94
REAL = 95, 30 .... GUESSED = 94, 27
REAL = 116, 75 .... GUESSED = 115, 73
REAL = 108, 93 .... GUESSED = 107, 92
REAL = 90, 24 .... GUESSED = 88, 22
REAL = 11, 79 .... GUESSED = 12, 80
REAL = 91, 105 .... GUESSED = 90, 104
REAL = 83, 75 .... GUESSED = 83, 73
REAL = 4, 9 .... GUESSED = 7, 10
REAL = 55, 120 .... GUESSED = 55, 120
REAL = 105, 119 .... GUESSED = 105, 118
REAL = 82, 89 .... GUESSED = 81, 89
REAL = 105, 92 .... GUESSED = 105, 91
REAL = 21, 20 .... G

REAL = 35, 89 .... GUESSED = 34, 88
REAL = 121, 98 .... GUESSED = 120, 97
REAL = 101, 91 .... GUESSED = 100, 90
REAL = 79, 93 .... GUESSED = 77, 92
REAL = 80, 35 .... GUESSED = 80, 35
REAL = 24, 92 .... GUESSED = 25, 92
REAL = 5, 97 .... GUESSED = 6, 97
REAL = 74, 63 .... GUESSED = 73, 63
REAL = 86, 56 .... GUESSED = 85, 55
REAL = 37, 123 .... GUESSED = 37, 122
REAL = 104, 94 .... GUESSED = 102, 93
REAL = 0, 75 .... GUESSED = 2, 75
REAL = 40, 65 .... GUESSED = 41, 65
REAL = 21, 82 .... GUESSED = 19, 82
REAL = 99, 39 .... GUESSED = 98, 38
REAL = 90, 45 .... GUESSED = 89, 45
REAL = 117, 73 .... GUESSED = 117, 71
REAL = 47, 80 .... GUESSED = 45, 78
REAL = 2, 48 .... GUESSED = 1, 47
REAL = 24, 98 .... GUESSED = 24, 97
REAL = 71, 84 .... GUESSED = 71, 83
REAL = 125, 35 .... GUESSED = 123, 35
REAL = 74, 0 .... GUESSED = 74, 2
REAL = 44, 84 .... GUESSED = 44, 83
REAL = 38, 102 .... GUESSED = 37, 100
REAL = 66, 118 .... GUESSED = 66, 118
REAL = 91, 84 .... GUESSED = 90, 83
REAL = 73, 79 .... G

REAL = 123, 50 .... GUESSED = 123, 49
REAL = 0, 110 .... GUESSED = 5, 110
REAL = 83, 36 .... GUESSED = 82, 36
REAL = 38, 48 .... GUESSED = 38, 49
REAL = 111, 119 .... GUESSED = 9, 17
REAL = 53, 49 .... GUESSED = 52, 48
REAL = 10, 0 .... GUESSED = 12, 3
REAL = 101, 83 .... GUESSED = 101, 81
REAL = 121, 117 .... GUESSED = 119, 115
REAL = 32, 118 .... GUESSED = 32, 116
REAL = 18, 29 .... GUESSED = 18, 29
REAL = 0, 53 .... GUESSED = 2, 53
REAL = 15, 34 .... GUESSED = 16, 34
REAL = 4, 55 .... GUESSED = 4, 55
REAL = 105, 25 .... GUESSED = 104, 24
REAL = 102, 4 .... GUESSED = 103, 4
REAL = 106, 117 .... GUESSED = 105, 116
REAL = 43, 59 .... GUESSED = 43, 59
REAL = 121, 99 .... GUESSED = 121, 99
REAL = 54, 58 .... GUESSED = 53, 57
REAL = 23, 34 .... GUESSED = 24, 34
REAL = 78, 124 .... GUESSED = 78, 124
REAL = 125, 52 .... GUESSED = 125, 52
REAL = 120, 13 .... GUESSED = 72, 72
REAL = 38, 3 .... GUESSED = 38, 4
REAL = 52, 45 .... GUESSED = 52, 44
REAL = 37, 49 .... GUESSED = 37, 50
REAL = 83, 7

REAL = 20, 58 .... GUESSED = 22, 58
REAL = 68, 57 .... GUESSED = 66, 56
REAL = 64, 5 .... GUESSED = 64, 7
REAL = 4, 80 .... GUESSED = 6, 81
REAL = 62, 101 .... GUESSED = 61, 101
REAL = 43, 50 .... GUESSED = 42, 50
REAL = 103, 113 .... GUESSED = 102, 112
REAL = 5, 116 .... GUESSED = 122, 122
REAL = 7, 113 .... GUESSED = 70, 70
REAL = 116, 110 .... GUESSED = 116, 111
REAL = 105, 28 .... GUESSED = 104, 27
REAL = 119, 91 .... GUESSED = 118, 90
REAL = 16, 33 .... GUESSED = 16, 34
REAL = 123, 84 .... GUESSED = 122, 83
REAL = 53, 76 .... GUESSED = 52, 74
REAL = 6, 47 .... GUESSED = 7, 47
REAL = 104, 44 .... GUESSED = 104, 43
REAL = 7, 14 .... GUESSED = 8, 14
REAL = 9, 121 .... GUESSED = 124, 124
REAL = 85, 32 .... GUESSED = 84, 30
REAL = 32, 6 .... GUESSED = 33, 8
REAL = 95, 52 .... GUESSED = 94, 52
REAL = 88, 66 .... GUESSED = 88, 65
REAL = 1, 67 .... GUESSED = 3, 68
REAL = 57, 90 .... GUESSED = 57, 89
REAL = 88, 111 .... GUESSED = 87, 110
REAL = 29, 22 .... GUESSED = 29, 23
REAL = 105, 89 .

In [ ]:
## test some stuff

In [136]:
for data in dataloader:
    conv1 = nn.Conv2d(1, 10, 5)
    pool = nn.MaxPool2d(2, 2)
    conv2 = nn.Conv2d(10, 10, 5)
    conv3 = nn.Conv2d(10, 10, 3)
    fc1 = nn.Linear(10 * 13 * 13, 250)
    fc2 = nn.Linear(250, 200)
    fc3 = nn.Linear(200, 100)

    x = data["image"]
    print(x.shape)
    x = pool(F.relu(conv1(x)))
    print(x.shape)
    x = pool(F.relu(conv2(x)))
    print(x.shape)
    x = pool(F.relu(conv3(x)))
    print(x.shape)
    x = x.view(-1, 10 * 13 * 13)
    x = F.relu(fc1(x))
    x = F.relu(fc2(x))
    x = fc3(x)
    print(x.shape)
    print(data["radius"])
    print(data["radius"].shape)
    break

torch.Size([16, 1, 128, 128])
torch.Size([16, 10, 62, 62])
torch.Size([16, 10, 29, 29])
torch.Size([16, 10, 13, 13])
torch.Size([16, 100])
tensor([[ 5],
        [82],
        [55],
        [31],
        [52],
        [58],
        [74],
        [33],
        [17],
        [49],
        [13],
        [80],
        [19],
        [46],
        [72],
        [18]])
torch.Size([16, 1])


In [140]:
torch.max(x, 1)[1][0]

tensor(9)